# Discover, Customize and Access NSIDC DAAC Data

This notebook will walk you through how to programmatically access data from the NASA National Snow and Ice Data Center Distributed Active Archive Center (NSIDC DAAC) using spatial and temporal filters, as well as how to request customization services including subsetting, reformatting, and reprojection. No Python experience is necessary; each code cell will prompt you with the information needed to configure your data request. The notebook will print the resulting API command that can be used in a command line, browser, or in Python as executed below.

1. CMR search to find MYD29E1D.6, SMAP (what data set?), and MODIS IST (MOD29) over East Siberian Sea bounding box over 3/23/19
2. Determine projections, formats, file structures, and parameters
3. Set customization options
4. Access data

### Import packages


In [3]:
import requests, getpass, socket, json, zipfile, io, math, os, shutil, pprint, re, time
from statistics import mean
from requests.auth import HTTPBasicAuth

### Input Earthdata Login credentials

An Earthdata Login account is required to access data from the NSIDC DAAC. If you do not already have an Earthdata Login account, visit http://urs.earthdata.nasa.gov to register.

In [4]:
# Earthdata Login credentials

uid = input('Earthdata Login user name: ')
pswd = getpass.getpass('Earthdata Login password: ')
email = input('Email address associated with Earthdata Login account: ')

Earthdata Login user name:  amy.steiker
Earthdata Login password:  ·········
Email address associated with Earthdata Login account:  amy.steiker@nsidc.org


### Select data set and determine version number

In [6]:
# Input data set ID (e.g. ATL07) of interest here, also known as "short name".

short_name = 'ATL07'

In [7]:
# Get json response from CMR collection metadata

params = {
    'short_name': short_name
}

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)

# Find all instances of 'version_id' in metadata and print most recent version number

versions = [el['version_id'] for el in results['feed']['entry']]
latest_version = max(versions)
print('The most recent version of ', short_name, ' is ', latest_version)

The most recent version of  ATL07  is  001


### Select time and area of interest

Data granules are returned based on a spatial bounding box and temporal range.

In [1]:
# Bounding Box spatial parameter in 'W,S,E,N' format

# Input lower left longitude in decimal degrees
LL_lon = '134.5'
# Input lower left latitude in decimal degrees
LL_lat = '68.18'
# Input upper right longitude in decimal degrees
UR_lon = '162.3'
# Input upper right latitude in decimal degrees
UR_lat = '85'

bounding_box = LL_lon + ',' + LL_lat + ',' + UR_lon + ',' + UR_lat
print(bounding_box)

134.5,68.18,162.3,85


In [9]:
#Input temporal range 

# Input start date in yyyy-MM-dd format
start_date = '2019-03-23'
# Input start time in HH:mm:ss format
start_time = '00:00:00'
# Input end date in yyyy-MM-dd format
end_date = '2019-03-23'
# Input end time in HH:mm:ss format
end_time = '23:59:59'

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'
print(temporal)

2019-03-23T00:00:00Z,2019-03-23T23:59:59Z


### Determine how many granules exist over this time and area of interest.

In [10]:
# Query number of granules (paging over results)

granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
params = {
    'short_name': short_name,
    'version': latest_version,
    'bounding_box': bounding_box,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1
}
granules = []
headers={'Accept': 'application/json'}
while True:
    response = requests.get(granule_search_url, params=params, headers=headers)
    results = json.loads(response.content)

    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    params['page_num'] += 1

print('There are', len(granules), 'granules of', short_name, 'version', latest_version, 'over my area and time of interest.')


There are 6 granules of ATL07 version 001 over my area and time of interest.


### Determine the average size of those granules as well as the total volume

In [11]:
granule_sizes = [float(granule['granule_size']) for granule in granules]

print(f'The average size of each granule is {mean(granule_sizes):.2f} MB and the total size of all {len(granules)} granules is {sum(granule_sizes):.2f} MB')


The average size of each granule is 242.30 MB and the total size of all 6 granules is 1453.80 MB


Note that subsetting, reformatting, or reprojecting can alter the size of the granules if those services are applied to your request.

### Select the subsetting, reformatting, and reprojection services enabled for your data set of interest.

The NSIDC DAAC supports customization services on many of our NASA Earthdata mission collections. Let's discover whether or not our data set has these services available. If services are available, we will also determine the specific service options supported for this data set and select which of these services we want to request. 

### Query the service capability endpoint to gather service information needed below

In [12]:
# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

# Create session to store cookie and pass credentials to capabilities url

session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))

root = ET.fromstring(response.content)

#collect lists with each service option

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

# variable subsetting
variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
variables_raw = [variables[i]['value'] for i in range(len(variables))]
variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
variable_vals = [v.replace(':', '/') for v in variables_join]

# reformatting
formats = [Format.attrib for Format in root.iter('Format')]
format_vals = [formats[i]['value'] for i in range(len(formats))]
format_vals.remove('')

# reformatting options that support reprojection
normalproj = [Projections.attrib for Projections in root.iter('Projections')]
normalproj_vals = []
normalproj_vals.append(normalproj[0]['normalProj'])
format_proj = normalproj_vals[0].split(',')
format_proj.remove('')
format_proj.append('No reformatting')

# reprojection options
projections = [Projection.attrib for Projection in root.iter('Projection')]
proj_vals = []
for i in range(len(projections)):
    if (projections[i]['value']) != 'NO_CHANGE' :
        proj_vals.append(projections[i]['value'])
        
# reformatting options that do not support reprojection
no_proj = [i for i in format_vals if i not in format_proj]

# SMAP-specific reprojection logic

#L1-L2 reprojection/reformatting options
if short_name == 'SPL1CTB' or 'SPL1CTB_E' or 'SPL2SMA' or 'SPL2SMP' or 'SPL2SMP_E' or 'SPL2SMAP': 
    format_proj = ['GeoTIFF', 'NetCDF4-CF', 'HDF-EOS5']
    no_proj = [i for i in format_vals if i not in format_proj]
elif short_name == 'SPL2SMAP_S' or 'SPL3SMA' or 'SPL3SMP' or 'SPL3SMP_E' or 'SPL3SMAP' or 'SPL3FTA' or 'SPL3FTP' or 'SPL3FTP_E' or 'SPL4SMAU' or 'SPL4SMGP' or 'SPL4SMLM' or 'SPL4CMDL': 
    format_proj = ['No reformatting', 'GeoTIFF', 'NetCDF4-CF', 'HDF-EOS5']
    no_proj = [i for i in format_vals if i not in format_proj]

### Select subsetting, reformatting, and reprojection service options, if available.

In [13]:
#print service information depending on service availability and select service options
    
if len(subagent) < 1 :
    print('No services exist for', short_name, 'version', latest_version)
    agent = 'NO'
    bbox = ''
    time_var = ''
    reformat = ''
    projection = ''
    projection_parameters = ''
    coverage = ''
else:
    agent = ''
    subdict = subagent[0]
    if subdict['spatialSubsetting'] == 'true':
        ss = input('Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n)')
        if ss == 'y': bbox = bounding_box
        else: bbox = ''
    if subdict['temporalSubsetting'] == 'true':
        ts = input('Subsetting by time, based on the temporal range inputted above, is available. Would you like to request this service? (y/n)')
        if ts == 'y': time_var = start_date + 'T' + start_time + ',' + end_date + 'T' + end_time 
        else: time_var = ''
    else: time_var = ''
    if len(format_vals) > 0 :
        print('These reformatting options are available:', format_vals)
        reformat = input('If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank.')
        # select reprojection options based on reformatting selection
        if reformat in format_proj and len(proj_vals) > 0 : 
            print('These reprojection options are available with your requested format:', proj_vals)
            projection = input('If you would like to reproject, copy and paste the reprojection option you would like (make sure to omit quotes, e.g. GEOGRAPHIC), otherwise leave blank.')
            # Enter required parameters for UTM North and South
            if projection == 'UTM NORTHERN HEMISPHERE' or projection == 'UTM SOUTHERN HEMISPHERE': 
                NZone = input('Please enter a UTM zone (1 to 60 for Northern Hemisphere; -60 to -1 for Southern Hemisphere):')
                projection_parameters = str('NZone:' + NZone)
            else: projection_parameters = ''
        else: 
            print('No reprojection options are supported with your requested format')
            projection = ''
            projection_parameters = ''
    else: 
        reformat = ''
        projection = ''
        projection_parameters = ''


Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n) y
Subsetting by time, based on the temporal range inputted above, is available. Would you like to request this service? (y/n) y


These reformatting options are available: ['TABULAR_ASCII', 'NetCDF4-CF', 'NetCDF-3']


If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank. 


No reprojection options are supported with your requested format


Because variable subsetting can include a long list of variables to choose from, we will decide on variable subsetting separately from the service options above.

In [14]:
# Select variable subsetting

if len(variable_vals) > 0:
        v = input('Variable subsetting is available. Would you like to subset a selection of variables? (y/n)')
        if v == 'y':
            print('The', short_name, 'variables to select from include:')
            print(*variable_vals, sep = "\n") 
            coverage = input('If you would like to subset by variable, copy and paste the variables you would like separated by comma (be sure to remove spaces and retain all forward slashes: ')
        else: coverage = ''

#no services selected
if reformat == '' and projection == '' and projection_parameters == '' and coverage == '' and time_var == '' and bbox == '':
    agent = 'NO'

Variable subsetting is available. Would you like to subset a selection of variables? (y/n) y


The ATL07 variables to select from include:
/ancillary_data
/ancillary_data/atlas_sdp_gps_epoch
/ancillary_data/control
/ancillary_data/data_end_utc
/ancillary_data/data_start_utc
/ancillary_data/end_cycle
/ancillary_data/end_delta_time
/ancillary_data/end_geoseg
/ancillary_data/end_gpssow
/ancillary_data/end_gpsweek
/ancillary_data/end_orbit
/ancillary_data/end_region
/ancillary_data/end_rgt
/ancillary_data/granule_end_utc
/ancillary_data/granule_start_utc
/ancillary_data/qa_at_interval
/ancillary_data/release
/ancillary_data/start_cycle
/ancillary_data/start_delta_time
/ancillary_data/start_geoseg
/ancillary_data/start_gpssow
/ancillary_data/start_gpsweek
/ancillary_data/start_orbit
/ancillary_data/start_region
/ancillary_data/start_rgt
/ancillary_data/version
/ancillary_data/coarse_surface_finding
/ancillary_data/coarse_surface_finding/bin_c
/ancillary_data/coarse_surface_finding/coarse_lb_wins
/ancillary_data/coarse_surface_finding/coarse_ub_wins
/ancillary_data/coarse_surface_find

If you would like to subset by variable, copy and paste the variables you would like separated by comma (be sure to remove spaces and retain all forward slashes:  


### Select data access configurations

The data request can be accessed asynchronously or synchronously. The asynchronous option will allow concurrent requests to be queued and processed without the need for a continuous connection. Those requested orders will be delivered to the specified email address, or they can be accessed programmatically as shown below. Synchronous requests will automatically download the data as soon as processing is complete. The granule limits differ between these two options:

Maximum granules per synchronous request = 100 

Maximum granules per asynchronous request = 2000 

We will set the access configuration depending on the number of granules requested. For requests over 2000 granules, we will produce multiple API endpoints for each 2000-granule order. Please note that synchronous requests may take a long time to complete depending on request parameters, so the number of granules may need to be adjusted if you are experiencing performance issues. The `page_size` parameter can be used to adjust this number. 

In [15]:
#Set NSIDC data access base URL
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'

#Set the request mode to asynchronous if the number of granules is over 100, otherwise synchronous is enabled by default
if len(granules) > 100:
    request_mode = 'async'
    page_size = 2000
else: 
    page_size = 100
    request_mode = 'stream'

#Determine number of orders needed for requests over 2000 granules. 
page_num = math.ceil(len(granules)/page_size)

print('There will be', page_num, 'total order(s) processed for our', short_name, 'request.')

There will be 1 total order(s) processed for our ATL07 request.


### Create the API endpoint 

Programmatic API requests are formatted as HTTPS URLs that contain key-value-pairs specifying the service operations that we specified above. The following command can be executed via command line, a web browser, or in Python below. 

In [16]:
#Create request parameter dictionary
param_dict = {'short_name': short_name, 
                  'version': latest_version, 
                  'temporal': temporal, 
                  'time': time_var, 
                  'bounding_box': bounding_box, 
                  'bbox': bbox, 
                  'format': reformat, 
                  'projection': projection, 
                  'projection_parameters': projection_parameters, 
                  'Coverage': coverage, 
                  'page_size': page_size, 
                  'request_mode': request_mode, 
                  'agent': agent, 
                  'email': email, }

#Remove blank key-value-pairs
param_dict = {k: v for k, v in param_dict.items() if v != ''}

#Convert to string
param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in param_dict.items())
param_string = param_string.replace("'","")

#Print API base URL + request parameters
endpoint_list = [] 
for i in range(page_num):
    page_val = i + 1
    API_request = api_request = f'{base_url}?{param_string}&page_num={page_val}'
    endpoint_list.append(API_request)

print(*endpoint_list, sep = "\n") 

https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL07&version=001&temporal=2019-03-23T00:00:00Z,2019-03-23T23:59:59Z&time=2019-03-23T00:00:00,2019-03-23T23:59:59&bounding_box=141.9,68.18,180,79.17&bbox=141.9,68.18,180,79.17&page_size=100&request_mode=stream&email=amy.steiker@nsidc.org&page_num=1


### Request data

We will now download data using the Python requests library. The data will be downloaded directly to this notebook directory in a new Outputs folder. The progress of each order will be reported.

In [17]:
# Create an output folder if the folder does not already exist.

path = str(os.getcwd() + '/Outputs')
if not os.path.exists(path):
    os.mkdir(path)

# Different access methods depending on request mode:

if request_mode=='async':
    # Request data service for each page number, and unzip outputs
    for i in range(page_num):
        page_val = i + 1
        print('Order: ', page_val)

    # For all requests other than spatial file upload, use get function
        request = session.get(base_url, params=param_dict)

        print('Request HTTP response: ', request.status_code)

    # Raise bad request: Loop will stop for bad response code.
        request.raise_for_status()
        print('Order request URL: ', request.url)
        esir_root = ET.fromstring(request.content)
        print('Order request response XML content: ', request.content)

    #Look up order ID
        orderlist = []   
        for order in esir_root.findall("./order/"):
            orderlist.append(order.text)
        orderID = orderlist[0]
        print('order ID: ', orderID)

    #Create status URL
        statusURL = base_url + '/' + orderID
        print('status URL: ', statusURL)

    #Find order status
        request_response = session.get(statusURL)    
        print('HTTP response from order response URL: ', request_response.status_code)

    # Raise bad request: Loop will stop for bad response code.
        request_response.raise_for_status()
        request_root = ET.fromstring(request_response.content)
        statuslist = []
        for status in request_root.findall("./requestStatus/"):
            statuslist.append(status.text)
        status = statuslist[0]
        print('Data request ', page_val, ' is submitting...')
        print('Initial request status is ', status)

    #Continue loop while request is still processing
        while status == 'pending' or status == 'processing': 
            print('Status is not complete. Trying again.')
            time.sleep(10)
            loop_response = session.get(statusURL)

    # Raise bad request: Loop will stop for bad response code.
            loop_response.raise_for_status()
            loop_root = ET.fromstring(loop_response.content)

    #find status
            statuslist = []
            for status in loop_root.findall("./requestStatus/"):
                statuslist.append(status.text)
            status = statuslist[0]
            print('Retry request status is: ', status)
            if status == 'pending' or status == 'processing':
                continue

    #Order can either complete, complete_with_errors, or fail:
    # Provide complete_with_errors error message:
        if status == 'complete_with_errors' or status == 'failed':
            messagelist = []
            for message in loop_root.findall("./processInfo/"):
                messagelist.append(message.text)
            print('error messages:')
            pprint.pprint(messagelist)

    # Download zipped order if status is complete or complete_with_errors
        if status == 'complete' or status == 'complete_with_errors':
            downloadURL = 'https://n5eil02u.ecs.nsidc.org/esir/' + orderID + '.zip'
            print('Zip download URL: ', downloadURL)
            print('Beginning download of zipped output...')
            zip_response = session.get(downloadURL)
            # Raise bad request: Loop will stop for bad response code.
            zip_response.raise_for_status()
            with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
                z.extractall(path)
            print('Data request', page_val, 'is complete.')
        else: print('Request failed.')
            
else:
    for i in range(page_num):
        page_val = i + 1
        print('Order: ', page_val)
        print('Requesting...')
        request = requests.get(base_url, params=param_dict)
        print('HTTP response from order response URL: ', request.status_code)
        request.raise_for_status()
        d = request.headers['content-disposition']
        fname = re.findall('filename=(.+)', d)
        dirname = os.path.join(path,fname[0].strip('\"'))
        print('Downloading...')
        open(dirname, 'wb').write(request.content)
        print('Data request', page_val, 'is complete.')
    
    # Unzip outputs
    for z in os.listdir(path): 
        if z.endswith('.zip'): 
            zip_name = path + "/" + z 
            zip_ref = zipfile.ZipFile(zip_name) 
            zip_ref.extractall(path) 
            zip_ref.close() 
            os.remove(zip_name) 


Order:  1
Requesting...
HTTP response from order response URL:  200
Downloading...
Data request 1 is complete.


### Finally, we will clean up the Output folder by removing individual order folders:

In [18]:
# Clean up Outputs folder by removing individual granule folders 

for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
    for name in dirs:
        os.rmdir(os.path.join(root, name))    

### To review, we have explored data availability and volume over a region and time of interest, discovered and selected data customization options, constructed an API endpoint for our request, and downloaded data directly to our local machine. You are welcome to request different data sets, areas of interest, and/or customization services by re-running the notebook or starting again at the 'Select a data set of interest' step above. 